In [1]:
import pandas as pd
import keras
import numpy as np

# Import CIFAR 10 dataset
from keras.datasets import cifar10

from keras.preprocessing.image import ImageDataGenerator

# Import Necessary CNN Building Blocks
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, UpSampling2D
from keras.layers.advanced_activations import LeakyReLU

from keras import backend as K
from keras import applications
from keras import callbacks
from keras import optimizers
from keras.utils import np_utils
import random

import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Requirements:
# !pip install keras
# !pip install tensorflow
# !pip install keras
# !pip install tensorflow_addons

# TODO: data augmentation validation

## Data preparation

In [3]:
# Load Dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

BATCH_SIZE = 32
INPUT_SHAPE = X_train.shape[1:]
NUM_CLASSES = 10

print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [4]:
# Normalize input data
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

# Encode target variable
y_train_ohe = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test_ohe = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

## Deep Learning pretrained models fitting

### VGG16

In [5]:
EPOCHS=5  # 50
BATCH_SIZE = 32
LR=1e-3
MOMENTUM=0.9

In [ ]:
vgg_base = tf.keras.applications.vgg16.VGG16(include_top=False,weights='imagenet',input_shape=INPUT_SHAPE)

vgg = Sequential()
vgg.add(vgg_base)
vgg.add(Flatten()) 
vgg.add(Dense(NUM_CLASSES, activation=('softmax')))
vgg.summary()

In [ ]:
vgg.compile(loss='binary_crossentropy',
            optimizer=tf.keras.optimizers.SGD(learning_rate=LR, momentum=MOMENTUM),
            metrics=['accuracy'])

In [ ]:
vgg_history = (
    vgg.fit(X_train,
            y_train, 
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            steps_per_epoch=X_train.shape[0]//BATCH_SIZE,
            validation_data=(X_test_vgg16, y_test_vgg16),
            validation_steps=X_test.shape[0]//BATCH_SIZE,
            verbose=1
           )
)

### Residual Network (ResNet)

In [6]:
LR = 2e-5
EPOCHS=5 # 10
INPUT_SHAPE = X_train.shape[1:]

In [7]:
input_tensor=tf.keras.Input(shape=INPUT_SHAPE)

In [8]:
res_base = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
    
resNet = Sequential()
resNet.add(res_base)
resNet.add(tf.keras.layers.Flatten())
resNet.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))

In [9]:
resNet.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.RMSprop(learning_rate=LR),
               metrics=['accuracy'])

In [10]:
resNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
resNet.fit(X_train,
           y_train_ohe,
           batch_size=BATCH_SIZE,
           epochs=EPOCHS,
           shuffle=True,
           verbose=1,
           validation_data=(X_test, y_test_ohe)
          )

Epoch 1/5
 235/1563 [===>..........................] - ETA: 50:49 - loss: 2.9357 - accuracy: 0.1806

## Models Evaluation

### Residual Network (ResNet)

In [ ]:
resNet.evaluate(X_test, y_test_ohe)

In [ ]:
y_pred_resnet = resNet.predict(X_test)
y_pred_resnet_classes = np.argmax(y_pred_resnet, axis=-1)

In [ ]:
pd.DataFrame(confusion_matrix(y_actual_test_classes, y_pred_resnet_classes))

In [ ]:
print(classification_report(y_actual_test_classes, y_pred_resnet_classes))

### VGG16

In [ ]:
# TODO
vggNet.evaluate(X_test_vgg16, y_test_vgg16)

In [ ]:
y_pred_test_vgg16 = vggNet.predict(X_test_vgg16)
y_pred_test_classes_vgg16 = np.argmax(y_pred_test_vgg16, axis=-1)

In [ ]:
pd.DataFrame(confusion_matrix(y_actual_test_classes, y_pred_test_classes_vgg16))

In [ ]:
print(classification_report(y_actual_test_classes, y_pred_test_classes_vgg16))